In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime


In [ ]:
crime_csv_path = "/Users/mignonduplessis/Downloads/Chicago_Crimes_2012_to_2017.csv"
weather_csv_path = "/Users/mignonduplessis/Documents/BootCampRep/row3/ChicagoDailyWeather.csv"
crime_type_path = "/Users/mignonduplessis/Documents/BootCampRep/row3/CrimeType.csv"

In [ ]:
weather_df = pd.read_csv(weather_csv_path)
crimes_df = pd.read_csv(crime_csv_path)
crime_type = pd.read_csv(crime_type_path)

In [ ]:
weather_df['DATE']=pd.to_datetime(weather_df['DATE'], format='%Y/%m/%d')

date1 = datetime.strptime('2012-01-01', '%Y-%m-%d')
date2 = datetime.strptime('2016-12-31', '%Y-%m-%d')

weather = weather_df.loc[(weather_df['DATE']>=date1) & (weather_df['DATE']<=date2)]
org_weather = weather.drop(['AWND', 'WDF2','STATION','FMTM', 'NAME','TAVG','TSUN', 'WDF5', 'WSF2', 'WSF5','WT01','WT02',
                            'WT03','WT04','WT05','WT08','WT09'], axis = 1)
new_weather = org_weather.rename(columns={"DATE": "Date"})
new_weather.head()

In [ ]:
new_weather.dtypes

In [ ]:
#org_weather = weather_df[['DATE', 'TAVG', 'TMAX', 'TMIN']].set_index(['DATE'])
#weather = org_weather.loc['2012-01-01':'2017-12-31']
#weather.head()

In [ ]:
crimes_df.dtypes

In [ ]:
crimes_df.columns
org_crimes = crimes_df.drop(['Ward', 'Community Area', 'FBI Code',
       'X Coordinate', 'Y Coordinate', 'Year', 'Updated On', 'Latitude',
       'Longitude', 'Location', 'Unnamed: 0', 'ID', 'Block', 'IUCR', 'Beat', 'District'], axis = 1)


org_crimes.head()


In [ ]:
#org_crimes.dtypes

In [ ]:
org_crimes['Date'] = pd.to_datetime(org_crimes['Date'])
org_crimes.head()

In [ ]:
org_crimes.dtypes

In [ ]:
org_crimes['Date'] = pd.to_datetime(org_crimes['Date']).dt.date
org_crimes.dtypes

In [ ]:
org_crimes['Date'] = pd.to_datetime(org_crimes['Date'])
org_crimes.dtypes

In [ ]:
## Merging 2 Datasets

weather_crime = pd.merge(org_crimes,new_weather, on="Date", how = 'outer')
weather_crime.dropna(how = 'any')
weather_crime.count()

In [ ]:
weather_crime = weather_crime.dropna(how = 'any')
weather_crime.count()

In [ ]:
weather_crime.head()

In [ ]:
#Get monthly high temperatures over the course of the 5 years
avg_df = weather_crime.loc[:,["Date", "TMAX"]]
avg_df['Year'] = pd.DatetimeIndex(avg_df["Date"]).year
avg_df['Month'] = pd.DatetimeIndex(avg_df["Date"]).month

Monthly_high = pd.DataFrame(avg_df.groupby(['Year', 'Month']).mean()['TMAX'])
Monthly_high.rename(columns = {'TMAX':'Monthly avg TMAX'}, inplace = True)

Monthly_high.head()
 

In [ ]:
#get the daily avg temp
avg_daily = weather_crime.loc[:,["Date", "TMAX", "TMIN"]]
maxtemp = weather_crime["TMAX"]
mintemp = weather_crime["TMIN"]
total_avg = maxtemp + mintemp / 2
avg_daily['Daily Average Temp'] = total_avg 
avg_daily.head()

In [ ]:
#5 Categories (Cold, Mild, Warm, Very Warm, Hot) 
#Below 32 Degrees Fahrenheit : Cold
#33 to 65 Degrees Fahrenheit: Mild
#66 to 75 Degrees Fahrenheit: Warm
#76 to 85 Degrees Fahrenheit: Very Warm
#Above 85 Degrees: Hot

#create function to filter through data

def f(row):

    if row['Daily Average Temp'] <= 32:

        val = 'Cold'

    elif row['Daily Average Temp'] <= 65:

        val = 'Mild'
        
    elif row['Daily Average Temp'] <= 75:

        val = 'Warm'
        
    elif row['Daily Average Temp'] <= 85:

        val = 'Very warm'

    else:

        val = 'Hot'

    return val



In [ ]:
#apply function & create new column for the return values
avg_daily['Category'] = avg_daily.apply(f, axis=1)
avg_daily.head()


In [ ]:
#4 Categories (Fall, Winter, Spring, Summer) 
#Fall: 09 to 11 
#Winter: 11 to 3
#Spring: 3 to 5
#Summer: 06 to 8


#adding year and month to average daily weather dataframe to create the season by month dataframe
season_df = avg_daily.loc[:,["Date", "TMAX","TMIN","Daily Average Temp","Category"]]
season_df['Year'] = pd.DatetimeIndex(season_df["Date"]).year
season_df['Month'] = pd.DatetimeIndex(season_df["Date"]).month

season_df.head()

In [ ]:
#create function to filter through data

def I(row):

    if row['Month'] <=3:

        val = 'Winter'

    elif row['Month'] <= 5:

        val = 'Spring'
        
    elif row['Month'] <= 8:

        val = 'Summer'
        
    elif row['Month'] <= 11:

        val = 'Fall'

    else:

        val = 'Winter'

    return val


In [ ]:
#apply function & create new column for the return values

season_df['Season'] = season_df.apply(I, axis=1)
season_df.head()

In [ ]:

crime_typedf = pd.merge(weather_crime,crime_type, on="Primary Type", how = 'outer')
crime_typedf.dropna(how = 'any')
crime_typedf.head()